In [1]:
import scipy                     # scientific computing and technical computing
import cv2                       # working with, mainly resizing, images
import numpy as np               # dealing with arrays
import glob                      # return a possibly-empty list of path names that match pathname
import os                        # dealing with directories
import pandas as pd              # providing data structures and data analysis tools
from PIL import Image
from scipy import ndimage
from skimage import io
%matplotlib inline
np.random.seed(1)

In [2]:
base_dir = os.getcwd()+"\\"

In [56]:
validation_img = io.imread('images_test.tiff')
validation_imgarray = np.array(validation_img)

In [57]:
df_test = pd.read_csv('answers.csv', header=None)
df_test = df_test.rename(columns={0:"imgID",1:"label",2:"c1x",3:"c1y",4:"c2x",5:"c2y"})

In [58]:
df_test.head(5)

,imgID,label,c1x,c1y,c2x,c2y
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0


Now loading trained model

In [59]:
# Initialize the parameters
confThreshold = 0.2  #Confidence threshold
nmsThreshold = 0.7   #Non-maximum suppression threshold
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

In [60]:
# Load names of classes
classesFile = base_dir+"yoloFiles\\"+"obj2.names"
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
 
    # Give the configuration and weight files for the model and load the network using them

    modelConfiguration = base_dir+"yoloFiles\\"+"yolo-obj2.cfg"
    modelWeights = base_dir+"yoloWeights\\"+"yolo-obj2_2000.weights"

    net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [61]:
def getSpotData(image):
    
    nspots=0
    _c1x=0
    _c1y=0
    _c2x=0
    _c2y=0
    
    (H, W) = image.shape[:2]
    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    
    boxes = []
    confidences = []
    classIDs = []
    
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > confThreshold:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    
    # ensure at least one detection exists
    if len(idxs) == 0:
        _c1x=0
        _c1y=0
        _c2x=0 
        _c2y=0
    elif len(idxs) == 1:
        nspots=1 
        i = idxs.flatten()[0]
        # extract the bounding box coordinates
        (x1, y1) = (boxes[i][0], boxes[i][1])
        (w1, h1) = (boxes[i][2], boxes[i][3])
        _c1x=int(x1+(w1/2)) 
        _c1y=int(y1+(h1/2))
        _c2x=0 
        _c2y=0
    elif len(idxs) == 2:
        nspots=2 
        i = idxs.flatten()[0]
        j = idxs.flatten()[1]
        # extract the bounding box coordinates for first spot
        (x1, y1) = (boxes[i][0], boxes[i][1])
        (w1, h1) = (boxes[i][2], boxes[i][3])
        _c1x=int(x1+(w1/2)) 
        _c1y=int(y1+(h1/2)) 
        # extract the bounding box coordinates for second spot
        (x2, y2) = (boxes[j][0], boxes[j][1])
        (w2, h2) = (boxes[j][2], boxes[j][3])
        _c2x=int(x2+(w2/2))
        _c2y=int(y2+(h2/2))
    else:
        print("Something is Wrong!!")
    return nspots, _c1x, _c1y, _c2x, _c2y

In [83]:
for idx, rows in df_test[22028:].iterrows():
    img = validation_imgarray[idx]
    nSpots, _c1x, _c1y, _c2x, _c2y = getSpotData(img)
    #print(idx, nSpots, _c1x, _c1y, _c2x, _c2y)
    df_test.label[idx] = nSpots
    df_test.c1x[idx] = _c1x
    df_test.c1y[idx] = _c1y
    df_test.c2x[idx] = _c2x
    df_test.c2y[idx] = _c2y

Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!
Something is Wrong!!


KeyboardInterrupt: 

In [84]:
df_test.to_csv(base_dir+"answers1.csv", header=False, index=False)